### Установка пакетов

In [ ]:
!pip install -U ogb

     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-cp37-none-any.whl size=7051 sha256=022483d856bd6cfb1369196e986ef52ee203cba64a0b230fab4013f8ea283228
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built littleutils


In [ ]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 2.5MB/s 
     |████████████████████████████████| 1.5MB 371kB/s 
     |████████████████████████████████| 215kB 3.9MB/s 
     |████████████████████████████████| 235kB 18.5MB/s 
     |████████████████████████████████| 2.2MB 18.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 


In [ ]:
!pip install dgl-cu101

     |████████████████████████████████| 36.2MB 73kB/s 


In [ ]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))

In [ ]:
install()

add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.10
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit
done
rdkit-2020.09.1 installation finished!


In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 808kB 11.2MB/s 
     |████████████████████████████████| 829kB 22.0MB/s 
     |████████████████████████████████| 112kB 37.6MB/s 
     |████████████████████████████████| 276kB 35.6MB/s 
     |████████████████████████████████| 645kB 33.6MB/s 
     |████████████████████████████████| 1.3MB 36.0MB/s 
     |████████████████████████████████| 143kB 37.4MB/s 
     |████████████████████████████████| 296kB 34.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=ef9be34e6b243dd8197416ab8a08fa49fb658e15d82a9a1aae089a7f6438ce10
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Обучение модели

In [ ]:
import argparse
import dgl
import numpy as np
import os
import random
import torch
import torch.optim as optim
import itertools

import rdkit
from ogb.lsc import PCQM4MDataset
from ogb.lsc import PygPCQM4MDataset
from ogb.lsc import DglPCQM4MDataset

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, tqdm_notebook
from dgl.data import DGLDataset

from pytorch_lightning.metrics.functional import r2score

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [ ]:
import rdkit
from ogb.lsc import PCQM4MDataset, PCQM4MEvaluator
from ogb.lsc import PygPCQM4MDataset
from ogb.lsc import DglPCQM4MDataset
from ogb.utils import smiles2graph

In [ ]:
!cp -r /content/gdrive/MyDrive/main/* /content

In [ ]:
dataset = PCQM4MDataset(only_smiles = True)

Downloaded 0.06 GB: 100%|██████████| 59/59 [00:01<00:00, 39.82it/s]


Extracting dataset/pcqm4m_kddcup2021.zip


In [ ]:
import gc
import tqdm

In [ ]:
def convert(dataset, index):
        smiles, y = dataset[index]
        graph_obj = smiles2graph(smiles)
        
        if len(graph_obj['node_feat']) > 255:
            graph_obj['edge_index'] = graph_obj['edge_index'].astype('uint16')
        else:
            graph_obj['edge_index'] = graph_obj['edge_index'].astype('uint8')

        graph_obj['node_feat'] = graph_obj['node_feat'].astype('uint8')
        graph_obj['edge_feat'] = graph_obj['edge_feat'].astype('uint8')
        
        if index % 10_000 == 0:
            gc.collect()

        return graph_obj

dataset_graph = [convert(dataset, i) for i in tqdm.notebook.tqdm(range(len(dataset)))]

In [ ]:
import pickle

In [ ]:
N=10
for i in range(N):
    with open(f'dataset_{i}.pkl', 'wb') as fp:
        count = len(dataset_graph) / N
        pickle.dump(dataset_graph[int(i * count) : int((i + 1) * count)], fp)

In [ ]:
!cp dataset_*.pkl /content/gdrive/MyDrive/dataset

In [ ]:
!cp /content/gdrive/MyDrive/dataset/dataset_*.pkl /content

In [ ]:
dataset_graph = []
N=10
for i in range(N):
    with open(f'dataset_{i}.pkl', 'rb') as fp:
        dataset_graph.append(pickle.load(fp))

dataset_graph = list(itertools.chain(*dataset_graph))

In [ ]:
len(dataset_graph), len(dataset)

(3803453, 3803453)

In [ ]:
import importlib
import main
import gnn

In [ ]:
importlib.reload(gnn)
importlib.reload(main)

<module 'main' from '/content/main.py'>

In [ ]:
class Dataset(DGLDataset):
    def __init__(self, dataset_smiles, dataset, indexes):
        self.dataset_smiles = dataset_smiles
        self.dataset = dataset
        self.indexes = indexes
        
    def __getitem__(self, index):
        graph_obj = self.dataset[int(self.indexes[index])]
        
        graph = dgl.graph((list(graph_obj['edge_index'][0].astype('int32')), 
                           list(graph_obj['edge_index'][1].astype('int32'))), 
                          num_nodes=graph_obj['num_nodes'])

        graph.ndata['feat'] = torch.from_numpy(graph_obj['node_feat'].astype('int32'))
        graph.edata['feat'] = torch.from_numpy(graph_obj['edge_feat'].astype('int32'))

        return graph, self.dataset_smiles[int(self.indexes[index])][1]
    
    def __len__(self):
        return len(self.indexes)

In [ ]:
def custom_main(dataset,
                seed=42,
                device=0,
                gnn_type='gin-virtual',
                JK='last',
                graph_pooling='sum',
                drop_ratio=0,
                num_layers=5,
                num_iter=5,
                emb_dim=600,
                train_subset=None,
                batch_size=256,
                epochs=100,
                num_workers=0,
                residual=True,
                log_dir='',
                checkpoint_dir='',
                save_test_dir=''):

    args={}
    args['seed']=seed
    args['device']=device
    args['gnn']=gnn_type
    args['graph_pooling']=graph_pooling
    args['drop_ratio']=drop_ratio
    args['num_layers']=num_layers
    args['num_iter']=num_iter
    args['emb_dim']=emb_dim
    args['train_subset']=train_subset
    args['batch_size']=batch_size
    args['epochs']=epochs
    args['num_workers']=num_workers
    args['residual']=residual
    args['log_dir']=log_dir
    args['checkpoint_dir']=checkpoint_dir
    args['save_test_dir']=save_test_dir
    
    print(args)

    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    random.seed(args['seed'])

    if torch.cuda.is_available():
        torch.cuda.manual_seed(args['seed'])
        device = torch.device("cuda:" + str(args['device']))
    else:
        device = torch.device("cpu")

    print(device)
    ### automatic dataloading and splitting
    #dataset = DglPCQM4MDataset(root='dataset/')

    dataset_smiles = PCQM4MDataset(only_smiles = True)

    # split_idx['train'], split_idx['valid'], split_idx['test']
    # separately gives a 1D int64 tensor
    split_idx = dataset_smiles.get_idx_split()

    ### automatic evaluator.
    evaluator = PCQM4MEvaluator()

    if args['train_subset']:
        subset_ratio = args['train_subset']
        subset_idx = torch.randperm(len(split_idx["train"]))[:int(subset_ratio * len(split_idx["train"]))]

        train_loader = DataLoader(Dataset(dataset_smiles, dataset, subset_idx), batch_size=args['batch_size'], shuffle=True,
                                  num_workers=args['num_workers'], collate_fn=main.collate_dgl)
    else:
        train_loader = DataLoader(Dataset(dataset_smiles, dataset, split_idx["train"]), batch_size=args['batch_size'], shuffle=True,
                                  num_workers=args['num_workers'], collate_fn=main.collate_dgl)

    valid_loader = DataLoader(Dataset(dataset_smiles, dataset, split_idx["valid"]), batch_size=args['batch_size'], shuffle=False,
                              num_workers=args['num_workers'], collate_fn=main.collate_dgl)

    if args['save_test_dir'] != '':
        test_loader = DataLoader(Dataset(dataset_smiles, dataset, split_idx["test"]), batch_size=args['batch_size'], shuffle=False,
                                 num_workers=args['num_workers'], collate_fn=main.collate_dgl)

    if args['checkpoint_dir'] != '':
        os.makedirs(args['checkpoint_dir'], exist_ok=True)

    shared_params = {
        'num_layers': args['num_layers'],
        'emb_dim': args['emb_dim'],
        'drop_ratio': args['drop_ratio'],
        'graph_pooling': args['graph_pooling'],
        'residual': args['residual'],
        'num_iter': args['num_iter']
    }

    if args['gnn'] == 'gin':
        model = gnn.GNN(gnn_type='gin', virtual_node=False, **shared_params).to(device)
    elif args['gnn'] == 'gin-virtual':
        model = gnn.GNN(gnn_type='gin', virtual_node=True, **shared_params).to(device)
    elif args['gnn'] == 'gcn':
        model = gnn.GNN(gnn_type='gcn', virtual_node=False, **shared_params).to(device)
    elif args['gnn'] == 'gcn-virtual':
        model = gnn.GNN(gnn_type='gcn', virtual_node=True, **shared_params).to(device)
    else:
        raise ValueError('Invalid GNN type')

    num_params = sum(p.numel() for p in model.parameters())
    print(f'#Params: {num_params}')

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    if args['log_dir'] != '':
        writer = SummaryWriter(log_dir=args['log_dir'])
        #writer.add_graph(model, next(iter(train_loader))[1])

    best_valid_mae = 1000

    if args['train_subset']:
        scheduler = StepLR(optimizer, step_size=300, gamma=0.25)
        args['epochs'] = 10
    else:
        scheduler = StepLR(optimizer, step_size=30, gamma=0.25)

    for epoch in range(1, args['epochs'] + 1):
        print("=====Epoch {}".format(epoch))
        print('Training...')
        train_mae = main.train(model, device, train_loader, optimizer)

        print('Evaluating...')
        valid_mae, valid_r2 = main.eval(model, device, valid_loader, evaluator)

        print({'Train': train_mae, 'Validation': valid_mae})

        if args['log_dir'] != '':
            writer.add_scalar('valid/mae', valid_mae, epoch)
            writer.add_scalar('valid/r2', valid_r2, epoch)
            writer.add_scalar('train/mae', train_mae, epoch)

        if valid_mae < best_valid_mae:
            best_valid_mae = valid_mae
            if args['checkpoint_dir'] != '' and epoch % 2 == 0:
                print('Saving checkpoint...')
                checkpoint = {'epoch': epoch, 'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'scheduler_state_dict': scheduler.state_dict(), 'best_val_mae': best_valid_mae,
                              'num_params': num_params}
                torch.save(checkpoint, os.path.join(args['checkpoint_dir'], 'checkpoint.pt'))

            if args['save_test_dir'] != '':
                print('Predicting on test data...')
                y_pred = main.test(model, device, test_loader)
                print('Saving test submission file...')
                evaluator.save_test_submission({'y_pred': y_pred}, args['save_test_dir'])

        scheduler.step()

        print(f'Best validation MAE so far: {best_valid_mae}')

    if args['log_dir'] != '':
        writer.close()

In [ ]:
custom_main(dataset_graph,
            graph_pooling='set2set',
            drop_ratio=0,
            num_layers=6,
            num_iter=1,
            emb_dim=300,
            epochs = 10,
            train_subset=1,
            batch_size=256,
            log_dir="logging_gin",
            checkpoint_dir='checking_gin',
            save_test_dir='submission_gin')

{'seed': 42, 'device': 0, 'gnn': 'gin-virtual', 'graph_pooling': 'set2set', 'drop_ratio': 0, 'num_layers': 5, 'num_iter': 1, 'emb_dim': 400, 'train_subset': 1, 'batch_size': 256, 'epochs': 10, 'num_workers': 0, 'residual': True, 'log_dir': 'logging_gin', 'checkpoint_dir': 'checking_gin', 'save_test_dir': 'submission_gin'}
cuda:0
#Params: 6204406
=====Epoch 1
Training...



Evaluating...



{'Train': 0.2471283660260879, 'Validation': 0.2896504668378829}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.2896504668378829
=====Epoch 2
Training...



Evaluating...


...


Saving test submission file...
Best validation MAE so far: 0.17412557893834849
=====Epoch 7
Training...



Evaluating...



{'Train': 0.13270119006013362, 'Validation': 0.17936331540374484}
Best validation MAE so far: 0.17412557893834849
=====Epoch 8
Training...



Evaluating...



{'Train': 0.12992387135446112, 'Validation': 0.19111002154648962}
Best validation MAE so far: 0.17412557893834849
=====Epoch 9
Training...


In [ ]:
!cp -r logging_gin /content/gdrive/MyDrive/
!cp -r checking_gin /content/gdrive/MyDrive/
!cp -r submission_gin /content/gdrive/MyDrive/

In [ ]:
custom_main(dataset_graph,
            graph_pooling='sum',
            drop_ratio=0,
            num_layers=5,
            num_iter=1,
            emb_dim=400,
            train_subset=0.1,
            batch_size=256,
            log_dir="logging_4",
            checkpoint_dir='checking_5',
            save_test_dir='submission_5')

{'seed': 42, 'device': 0, 'gnn': 'gin-virtual', 'graph_pooling': 'sum', 'drop_ratio': 0, 'num_layers': 5, 'num_iter': 1, 'emb_dim': 400, 'train_subset': 0.1, 'batch_size': 256, 'epochs': 100, 'num_workers': 0, 'residual': True, 'log_dir': 'logging_4', 'checkpoint_dir': 'checking_5', 'save_test_dir': 'submission_5'}
cuda:0
#Params: 2997606
=====Epoch 1
Training...



Evaluating...



{'Train': 1.2060507432252419, 'Validation': 0.9965707995359875}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.9965707995359875
=====Epoch 2
Training...



Evaluating...


...


Evaluating...



{'Train': 0.3880998222767806, 'Validation': 0.40425513622370624}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.40425513622370624
=====Epoch 10
Training...



Evaluating...



{'Train': 0.36713995577908365, 'Validation': 0.46953609054412704}
Best validation MAE so far: 0.40425513622370624


In [ ]:
!cp -r logging_5 /content/gdrive/MyDrive/
!cp -r checking_5 /content/gdrive/MyDrive/
!cp -r submission_5 /content/gdrive/MyDrive/

cp: cannot stat 'logging_5': No such file or directory


In [ ]:
custom_main(dataset_graph,
            graph_pooling='set2set',
            drop_ratio=0,
            num_layers=1,
            num_layers=5,
            emb_dim=400,
            train_subset=0.1,
            batch_size=256,
            log_dir="logging_4",
            checkpoint_dir='checking_4',
            save_test_dir='submission_4')

{'seed': 42, 'device': 0, 'gnn': 'gin-virtual', 'graph_pooling': 'set2set', 'drop_ratio': 0, 'num_layers': 5, 'emb_dim': 400, 'train_subset': 0.1, 'batch_size': 256, 'epochs': 100, 'num_workers': 0, 'residual': True, 'log_dir': 'logging_1', 'checkpoint_dir': 'checking_1', 'save_test_dir': 'submission_1'}
cuda:0
#Params: 6204406
=====Epoch 1
Training...



Evaluating...



{'Train': 0.42107979252057914, 'Validation': 0.33294821735100244}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.33294821735100244
=====Epoch 2
Training...



Evaluating...


...


Evaluating...



{'Train': 0.19168171567075393, 'Validation': 0.22033330329807302}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.22033330329807302
=====Epoch 10
Training...



Evaluating...



{'Train': 0.18935914938690282, 'Validation': 0.22284111605698967}
Best validation MAE so far: 0.22033330329807302


In [ ]:
custom_main(graph_pooling='set2set',
           drop_ratio=0.3,
           num_layers=5,
           emb_dim=400,
           train_subset=0.1,
           batch_size=256,
           log_dir="logging_1",
           checkpoint_dir='checking_1',
           save_test_dir='submission_1')

{'seed': 42, 'device': 0, 'gnn': 'gin-virtual', 'graph_pooling': 'set2set', 'drop_ratio': 0.3, 'num_layers': 5, 'emb_dim': 400, 'train_subset': 0.1, 'batch_size': 256, 'epochs': 100, 'num_workers': 0, 'residual': True, 'log_dir': 'logging_1', 'checkpoint_dir': 'checking_1', 'save_test_dir': 'submission_1'}
cuda:0
#Params: 6204406
=====Epoch 1
Training...



Evaluating...



{'Train': 0.44267863060245993, 'Validation': 0.3776600811576754}
Saving checkpoint...
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.3776600811576754
=====Epoch 2
Training...



Evaluating...


...


Saving test submission file...
Best validation MAE so far: 0.24476570625015512
=====Epoch 9
Training...



Evaluating...



{'Train': 0.21767751381677738, 'Validation': 0.26860140578023606}
Best validation MAE so far: 0.24476570625015512
=====Epoch 10
Training...



Evaluating...



{'Train': 0.2139916999249899, 'Validation': 0.24600969989021904}
Best validation MAE so far: 0.24476570625015512


In [ ]:
!cp -r logging_1 /content/gdrive/MyDrive/
!cp -r checking_1 /content/gdrive/MyDrive/
!cp -r submission_1 /content/gdrive/MyDrive/

In [ ]:
custom_main(dataset_graph,
           graph_pooling='set2set',
           drop_ratio=0,
           num_layers=3,
           emb_dim=200,
           train_subset=0.1,
           batch_size=256,
           log_dir="logging_2",
           checkpoint_dir='checking_2',
           save_test_dir='submission_2')

{'seed': 42, 'device': 0, 'gnn': 'gin-virtual', 'graph_pooling': 'set2set', 'drop_ratio': 0, 'num_layers': 3, 'emb_dim': 200, 'train_subset': 0.1, 'batch_size': 256, 'epochs': 100, 'num_workers': 0, 'residual': True, 'log_dir': 'logging_2', 'checkpoint_dir': 'checking_2', 'save_test_dir': 'submission_2'}
cuda:0
#Params: 1252204
=====Epoch 1
Training...



Evaluating...



{'Train': 0.35161003560579124, 'Validation': 0.3107551008332873}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.3107551008332873
=====Epoch 2
Training...



Evaluating...


...


Evaluating...



{'Train': 0.18855499015385363, 'Validation': 0.22599879319608637}
Predicting on test data...



Saving test submission file...
Best validation MAE so far: 0.22599879319608637
=====Epoch 10
Training...



Evaluating...



{'Train': 0.1835887026260881, 'Validation': 0.22605792301935507}
Best validation MAE so far: 0.22599879319608637


In [ ]:
!cp -r logging_2 /content/gdrive/MyDrive/
!cp -r checking_2 /content/gdrive/MyDrive/
!cp -r submission_2 /content/gdrive/MyDrive/

In [ ]:
custom_main(dataset_graph,
           graph_pooling='sum',
           drop_ratio=0,
           num_layers=5,
           emb_dim=400,
           train_subset=0.1,
           batch_size=256,
           log_dir="logging_3",
           checkpoint_dir='checking_3',
           save_test_dir='submission_3')

In [ ]:
!cp -r logging_3 /content/gdrive/MyDrive/
!cp -r checking_3 /content/gdrive/MyDrive/
!cp -r submission_3 /content/gdrive/MyDrive/